In [1]:
#import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from pycoingecko import CoinGeckoAPI
import seaborn as sns
import datetime as dt
import ast
pd.set_option('display.max_columns', None) 
cg = CoinGeckoAPI()
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

## 01 - Capitalizacion

In [2]:
#Generamos el Dataset para ver los mayor capitalizacion de las monedas
mayor_cap = pd.DataFrame.from_dict(cg.get_coins_markets(vs_currency = 'usd')).reset_index(drop=True)
mayor_cap=mayor_cap.sort_values(by="market_cap", ascending=False)
mayor_cap=mayor_cap.iloc[0:10]
mayor_cap=mayor_cap[["symbol","name","market_cap"]]
mayor_cap.to_csv("../00 - DATA/mayor_cap.csv",index=False)


In [3]:
# Generamos una una funcion para extraer los datos que estan en UNIX y convertirlos a fecha.-

def sacar(x):
    x = x.translate(str.maketrans('', '', "["))
    x = x.translate(str.maketrans('', '', "]"))
    fila=x
    return fila

#Llamo a la API y genero la transformacion

def cambio(moneda):
    globalData =  cg.get_coin_market_chart_by_id(id = moneda, 
                               vs_currency = 'usd',
                               days = '1825')
                               
    lista = pd.DataFrame.from_dict(globalData ).reset_index(drop=True)
    moneda=str(moneda)
    lista=lista["market_caps"]
    df=[]
    for i in lista:
        i=str(i)
        a=sacar(i)
        df.append(a)
    #En esta funcion separo y convierto los datos en varibles para poder trabajarcon ellos
    df=pd.DataFrame(df)
    df[['Fecha', 'Market_Cap']] = df[0].str.split(',', expand=True)
    df=df[['Fecha', 'Market_Cap']]
    df["Fecha"]=df["Fecha"].astype(float) 
    #Convierto en Fecha los Datos en UNIX
    df["Fecha"]=df["Fecha"].apply(lambda x: dt.datetime.fromtimestamp(x/1000).strftime('%m-%d-%Y' ))
    df["Fecha"]=pd.to_datetime(df["Fecha"])
    df.insert(0, 'Moneda',moneda)
    #Convierto los datos Faltantes en la media de la variable anterior y la posterior
    df["Market_Cap"]=df["Market_Cap"].replace(' None', float('nan'))
    df["Market_Cap"]=df["Market_Cap"].fillna(method='ffill').fillna(method='bfill')
    df["Market_Cap"]=df["Market_Cap"].astype(float)
        
    return df


In [4]:
#Genero los dataframe de todas las monedas y los concateno en un solo CSV

bitcoin=cambio("bitcoin")
ethereum=cambio("ethereum")
tether=cambio("tether")
binancecoin=cambio("binancecoin")
ripple=cambio("ripple")
usdcoin=cambio("usd-coin")
stakedether=cambio("staked-ether")
cardano=cambio("cardano")
dogecoin=cambio("dogecoin")
solana=cambio("solana")
Moneda_cap_hist=pd.concat([bitcoin,ethereum,tether,binancecoin,ripple,usdcoin,stakedether,cardano,dogecoin,solana], axis=0)
Moneda_cap_hist['Fecha'] = pd.to_datetime(Moneda_cap_hist['Fecha'])
Moneda_cap_hist.to_csv("../00 - DATA/Moneda_cap_hist.csv",index=False)

## 02 - Precio Historico de todas las monedas

In [5]:
#Genero funcion llamando  a la API para obtener todos los resultados y tranformar los de UNIX

def Precio(moneda):
    globalData =  cg.get_coin_market_chart_by_id(id = moneda, 
                               vs_currency = 'usd',
                               days = '1825')
                               
    lista = pd.DataFrame.from_dict(globalData ).reset_index(drop=True)
    moneda=str(moneda)
    lista=lista["prices"]
    df=[]
    for i in lista:
        i=str(i)
        a=sacar(i)
        df.append(a)
    
    df=pd.DataFrame(df)
    df[['Fecha', 'prices']] = df[0].str.split(',', expand=True)
    df=df[['Fecha', 'prices']]
    df["Fecha"]=df["Fecha"].astype(float) 
    df["Fecha"]=df["Fecha"].apply(lambda x: dt.datetime.fromtimestamp(x/1000).strftime('%m-%d-%Y' ))
    df["Fecha"]=pd.to_datetime(df["Fecha"])
    df.insert(0, 'Moneda',moneda)
    df["prices"]=df["prices"].replace(' None', float('nan'))
    df["prices"]=df["prices"].fillna(method='ffill').fillna(method='bfill')
    df["prices"]=df["prices"].astype(float).round(2)
        
    return df

In [6]:
#Genero los dataframe de todas las monedas y los concateno en un solo CSV
bitcoin=Precio("bitcoin")
ethereum=Precio("ethereum")
tether=Precio("tether")
binancecoin=Precio("binancecoin")
ripple=Precio("ripple")
usdcoin=Precio("usd-coin")
stakedether=Precio("staked-ether")
cardano=Precio("cardano")
dogecoin=Precio("dogecoin")
solana=Precio("solana")
Precio_Total=pd.concat([bitcoin,ethereum,tether,binancecoin,ripple,usdcoin,stakedether,cardano,dogecoin,solana])
Precio_Total.to_csv("../00 - DATA/Precio_Total.csv",index=False)


In [7]:
Precio_Total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16776 entries, 0 to 1231
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Moneda  16776 non-null  object        
 1   Fecha   16776 non-null  datetime64[ns]
 2   prices  16776 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 524.2+ KB


In [8]:
#Genero los csv de todas monedas segun el parametro de  precio

moneda_precio_alto=pd.concat([bitcoin], axis=0)
moneda_precio_alto.to_csv("../00 - DATA/moneda_precio_alto.csv",index=False)
moneda_precio_medio=pd.concat([stakedether,ethereum], axis=0)
moneda_precio_medio.to_csv("../00 - DATA/moneda_precio_medio.csv",index=False)
moneda_precio_Bajo=pd.concat([binancecoin,solana], axis=0)
moneda_precio_Bajo.to_csv("../00 - DATA/moneda_precio_Bajo.csv",index=False)
moneda_precio_estable=pd.concat([tether,usdcoin,ripple,cardano,dogecoin], axis=0)
moneda_precio_estable.to_csv("../00 - DATA/moneda_precio_estable.csv",index=False)

In [9]:



exchgList = cg.get_exchanges_list(per_page = 250, page = 1)
exchgDataFrame = pd.DataFrame.from_dict(exchgList
                              ).sort_values('trade_volume_24h_btc_normalized',
                                            ascending = False
                              ).reset_index(drop=True)
coinsMktDataByCategory=exchgDataFrame[["name","country","trade_volume_24h_btc_normalized"]]

Exchange=coinsMktDataByCategory.head(5)
Exchange.to_csv("Exchange.csv",index=False)


## 3 - Volumen historico

In [10]:
# Generamos la funcion para extraer los volumnes historico

def volumen(moneda):
    globalData =  cg.get_coin_market_chart_by_id(id = moneda, 
                               vs_currency = 'usd',
                               days = '1825')
                               
    lista = pd.DataFrame.from_dict(globalData ).reset_index(drop=True)
    moneda=str(moneda)
    lista=lista["prices"]
    df=[]
    for i in lista:
        i=str(i)
        a=sacar(i)
        df.append(a)
    #En esta funcion separo y convierto los datos en varibles para poder trabajarcon ellos
    df=pd.DataFrame(df)
    df[['Fecha', 'total_volumes']] = df[0].str.split(',', expand=True)
    df=df[['Fecha', 'total_volumes']]
    df["Fecha"]=df["Fecha"].astype(float) 
    #Convierto en Fecha los Datos en UNIX
    df["Fecha"]=df["Fecha"].apply(lambda x: dt.datetime.fromtimestamp(x/1000).strftime('%m-%d-%Y' ))
    df["Fecha"]=pd.to_datetime(df["Fecha"])
    df.insert(0, 'Moneda',moneda)
    #Convierto los datos Faltantes en la media de la variable anterior y la posterior
    df["total_volumes"]=df["total_volumes"].replace(' None', float('nan'))
    df["total_volumes"]=df["total_volumes"].fillna(method='ffill').fillna(method='bfill')
    df["total_volumes"]=df["total_volumes"].astype(float).round(2)
        
    return df

In [11]:
# Genero los dataframe de las monedas y Genero el csv.-

bitcoin=volumen("bitcoin")
ethereum=volumen("ethereum")
tether=volumen("tether")
binancecoin=volumen("binancecoin")
ripple=volumen("ripple")
usdcoin=volumen("usd-coin")
stakedether=volumen("staked-ether")
cardano=volumen("cardano")
dogecoin=volumen("dogecoin")
solana=volumen("solana")
volumen=pd.concat([bitcoin,ethereum,tether,binancecoin,ripple,usdcoin,stakedether,cardano,dogecoin,solana], axis=0)
volumen.to_csv("../00 - DATA/volumen.csv",index=False)

## 4 - Exchange 

In [12]:


exchgList = cg.get_exchanges_list(per_page = 250, page = 1)
exchgDataFrame = pd.DataFrame.from_dict(exchgList
                              ).sort_values('trade_volume_24h_btc_normalized',
                                            ascending = False
                              ).reset_index(drop=True)
coinsMktDataByCategory=exchgDataFrame[["name","country","trade_volume_24h_btc_normalized"]]

Exchange=coinsMktDataByCategory.head(5)
Exchange.to_csv("../00 - DATA/Exchange.csv",index=False)
